In [ ]:
import xarray as xr
import rioxarray
import rasterio
import numpy as np
from rasterio.enums import Resampling
from rasterio.fill import fillnodata

# ==== User settings ====
input_nc = r"C:\Users\Ankit\Datasets_Forest_fire\compressed_cleaned_era5_2015_2016.nc"
output_tif = r"C:\Users\Ankit\Datasets_Forest_fire\ERA5_30m_stack_filled_final.tif"
variables_to_process = ['t2m', 'd2m', 'u10', 'v10', 'tp']
target_resolution = 30  # meters
target_crs = "EPSG:32644"  # UTM Zone (adjust if needed)

# ---- STEP 1: Read and process ERA5 variables ----
ds = xr.open_dataset(input_nc)
print("Dataset dimensions:", ds.dims)
print("Variables:", list(ds.data_vars))

processed_bands = []

for var in variables_to_process:
    print(f"Processing variable: {var}")
    da = ds[var]

    # Handle time aggregation differently for each var
    if "time" in da.dims:
        if var == "tp":
            da = da.sum(dim="time")  # total precipitation
        else:
            da = da.max(dim="time")  # retain extremes
    elif "valid_time" in da.dims:
        if var == "tp":
            da = da.sum(dim="valid_time")
        else:
            da = da.max(dim="valid_time")
    else:
        raise ValueError(f"{var} has no time dimension!")

    # Attach CRS
    da = da.rio.write_crs("EPSG:4326")

    # Reproject to UTM @ 30m
    da_utm = da.rio.reproject(
        target_crs,
        resolution=target_resolution,
        resampling=Resampling.bilinear
    )

    processed_bands.append(da_utm)

# ---- STEP 2: Stack into multi-band array ----
stacked = xr.concat(processed_bands, dim="band")
stacked = stacked.assign_coords(band=range(1, len(variables_to_process) + 1))

# Convert to rasterio-compatible array
arr = stacked.values.astype("float32")

# ---- STEP 3: Fill nodata values band-wise ----
filled_bands = []
for i in range(arr.shape[0]):  # loop over bands
    band_data = arr[i, :, :]
    mask = np.isnan(band_data)
    if mask.any():
        print(f"Filling nodata in band {i+1} ...")
        band_filled = fillnodata(band_data, mask=mask, max_search_distance=50)
    else:
        band_filled = band_data
    # Replace any NaN with -9999
    band_filled = np.nan_to_num(band_filled, nan=-9999).astype("float32")
    filled_bands.append(band_filled)

filled_arr = np.stack(filled_bands, axis=0)

# ---- STEP 4: Write final GeoTIFF ----
profile = stacked.rio.profile
profile.update(dtype="float32", count=len(variables_to_process), nodata=-9999, compress="LZW")

with rasterio.open(output_tif, "w", **profile) as dst:
    dst.write(filled_arr)

print(f"✅ Final multi-band raster (with nodata filled) saved at: {output_tif}")


Dataset dimensions: FrozenMappingWarningOnValuesAccess({'valid_time': 17544, 'latitude': 13, 'longitude': 17})
Variables: ['t2m', 'd2m', 'u10', 'v10', 'tp']
Processing variable: t2m
Processing variable: d2m
Processing variable: u10
